In [ ]:
import ee, os, geemap
ee.Initialize()
Map = geemap.Map()

# Image information and metadata

In [ ]:
# // Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# // Get information about the bands as a list.
bandNames = image.bandNames()
# print('Band names: ', bandNames); // ee.List of band names

# // Get projection information from band 1.
b1proj = image.select('B1').projection()
# print('Band 1 projection: ', b1proj); // ee.Projection object

# // Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
# print('Band 1 scale: ', b1scale); // ee.Number

# // Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties: ', properties) # ee.List of metadata properties

# // Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
# print('CLOUD_COVER: ', cloudiness); // ee.Number

# // Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'))
# print('Timestamp: ', date); // ee.Date
    

# Mathematical operations

In [ ]:
# // Load a 5-year Landsat 7 composite 1999-2003.
var landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

# // Compute NDVI.
var ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3'))
  .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

In [ ]:
# // Load a Landsat 8 image.
var image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# // Compute the EVI using an expression.
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')})

Map.centerObject(image, 9)
Map.addLayer(evi, {min: -1, max: 1, palette: ['FF0000', '00FF00']})

Relational, conditional operations

In [ ]:
# // Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# // Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# // Create a binary layer using logical operations.
bare = ndvi.lt(0.2).and(ndwi.lt(0))

# // Mask and display the binary layer.
Map.setCenter(-122.3578, 37.7726, 12)
Map.setOptions('satellite')
Map.addLayer(bare.updateMask(bare), {}, 'bare')
# ------------------------------------------------------------------------------------------
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# ------------------------------------------------------------------------------------------
# // Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# // Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# // Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {min: 0, max: 3, palette: palette}, 'development zones')
    